In [1]:
import graphviz
import tensorflow as tf 
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import Adam
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy, BoltzmannGumbelQPolicy, GreedyQPolicy
from keras.callbacks import TensorBoard
from IPython.display import SVG 
from keras.utils.vis_utils import model_to_dot
from rl.callbacks import ModelIntervalCheckpoint
class DRL(DQNAgent):
    def __init__(self, history_length, time_var, time_data_length, not_time_data_length, nb_actions, file_nm=None, visualize=False):

        self._history_length = history_length
        self.time_var = time_var
        self._time_data_length = time_data_length
        self._not_time_data_length = not_time_data_length
        self._nb_actions = nb_actions
        self._visualize = visualize
        self._file_nm = file_nm
#         config = tf.ConfigProto( gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.20))
        config = tf.ConfigProto( gpu_options = tf.GPUOptions(allow_growth=True))
        session = tf.Session(config=config)
#         self.call1 = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
        if self._file_nm:
#             self.call2 = ModelCheckByTrainAccRe(filepath=file_nm + '_{month_return_diff:.5f}.h5', 
#                                             monitor='month_return_diff', verbose=2, mode='max',min_save_length=4000)
#             self.call2 = Earlystop(filepath=self._file_nm + '_{accumulate_return:.5f}.h5',patience=2)
#             self.call2 = Earlystop()
            self.call3 = TrainEpisodeLogger(filepath=self._file_nm+ '.csv')
#             self.call4 = ModelIntervalCheckpoint(filepath=file_nm+'_{step}'+'.h5', interval=50000, verbose=1)
    def NN_model(self):
#         if self._not_time_data_length>2:
        input_shape = (1,self._time_data_length+self._not_time_data_length)
        input_sq = Input(shape=input_shape)
        x1 = Lambda(split_data,arguments={'index':[0,self._time_data_length]})(input_sq)
        x1 = Reshape((self.time_var,self._history_length))(x1)
        x1 = Permute((2,1))(x1)
        x1 = CuDNNGRU(32, return_sequences=True)(x1)
        x1 = CuDNNGRU(16, return_sequences=False)(x1)
        x2 = Lambda(split_data,arguments={'index':[self._time_data_length,self._time_data_length+self._not_time_data_length]})(input_sq)
        x2 = Flatten()(x2)
        x2 = Dense(32, activation='selu')(x2)
        x2 = Dense(16, activation='selu')(x2)            
        x3 = Lambda(split_data,arguments={'index':[self._time_data_length,self._time_data_length+2]})(input_sq)
        x3 = Flatten()(x3)
        m1 = Concatenate(axis=1)([x1,x2]) 
        m1 = Dense(32, activation='selu')(m1)
        m1 = Dense(16, activation='selu')(m1)
        m1 = Dense(8, activation='selu')(m1)
        m1 = Concatenate(axis=1)([m1,x3]) 
        m1 = Dense(self._nb_actions, activation='linear')(m1)
        model = Model(input_sq, m1)
#             model.summary()
#         else :
#             input_shape = (1,self._time_data_length+self._not_time_data_length)
#             input_sq = Input(shape=input_shape)
#             x1 = Lambda(split_data,arguments={'index':[0,self._time_data_length]})(input_sq)
#             x1 = Reshape((self.time_var,self._history_length))(x1)
#             x1 = Permute((2,1))(x1)
#             x1 = CuDNNGRU(32, return_sequences=True)(x1)
#             x1 = CuDNNGRU(16, return_sequences=False)(x1)
#             x3 = Lambda(split_data,arguments={'index':[self._time_data_length-1,self._time_data_length+1]})(input_sq)
#             x3 = Flatten()(x3) 
#             m1 = Dense(32, activation='selu')(x1)
#             m1 = Dense(16, activation='selu')(m1)
#             m1 = Dense(8, activation='selu')(m1)
#             m1 = Concatenate(axis=1)([m1,x3]) 
#             m1 = Dense(self._nb_actions, activation='linear')(m1)
#             model = Model(input_sq, m1)
#             model.summary()
        return model
    
    def visualize_model(self):
        return SVG(model_to_dot(self.NN_model(),show_shapes=True).create(prog='dot', format='svg'))
      
    def RL_method(self, env, check_env, step, memory_length, policy, nb_steps_warmup, batch_size, gamma, 
                  target_model_update,train_interval,alpha,beta,beta_annealed, reword_step=1):
        #memory = SequentialMemory(limit=memory_length, window_length=1)
        memory = PrioritizedMemory(limit=memory_length, alpha=alpha, start_beta=beta, end_beta=1., steps_annealed=beta_annealed, window_length=1)
        dqn = DQNAgent(model=self.NN_model(), nb_actions=self._nb_actions, memory=memory, nb_steps_warmup=nb_steps_warmup,
                       batch_size=batch_size, gamma=gamma, enable_dueling_network=True, dueling_type='avg',
                       target_model_update=target_model_update, policy=policy,train_interval=train_interval)
        dqn.compile(Adam(lr=1e-3), metrics=['mae'])
        if self._file_nm:
            dqn.fit(env,check_env, file_nm=self._file_nm + '_{}.h5', patience=6,check_interval=500 ,\
                    nb_steps=step, visualize=False, verbose=2,callbacks=[self.call3])
#             dqn.save_weights(save_nm, overwrite=True)
        return dqn
    

Using TensorFlow backend.


NameError: name 'DQNAgent' is not defined